In [2]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../data/'
folder = 'sp_matrix_tag_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [3]:
df = pd.read_pickle(os.path.join(dir_, folder, 'pop_d_' + file_name))
train = pd.read_pickle(os.path.join(dir_, folder, 'train_' + file_name))
train_ = train[['uid', 'tid', 'rating', 'count','pop']]

In [3]:
train_x = sp.load_npz(os.path.join(dir_, folder, 'pop_d_train_x_' + file_name + '.npz'))
train_y = np.loadtxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')
test_x_sp = sp.load_npz(os.path.join(dir_, folder, 'pop_d_test_x_' + file_name + '.npz'))
test_y_sp = np.loadtxt(os.path.join(dir_, folder, 'test_y_' + file_name[:-3] + 'csv'), delimiter=',')

In [4]:
train_x.shape, len(train_y), test_x_sp.shape, len(test_y_sp)

((362677, 39375), 362677, (90670, 39375), 90670)

In [5]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 19 + 1
tid_start = unique_users
tag_start = unique_users + unique_tracks
pop_start = unique_users + unique_tracks + 19
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 218
Tag start position : 39355
Length of vector : 39375
----------------------------------------
Number of unique users : 218
Number of unique tracks : 39137


In [6]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [7]:
x_test = []
for i in tqdm(range(unique_users)):
    user = train_[train_['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)
    

In [8]:
i = 0
v = []
sp_rows = []
sp_cols = []
for row in tqdm(x_test):
    sp_rows.append(i)
    sp_cols.append(row[0])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    v.append(1)
    
    t = df[df['tid']==row[1]].reset_index()
    tags = t['tags'].iloc[0]
    sp_rows.append(i)
    sp_cols.append(tag_start + tags)
    v.append(1)
    
    pop = t['pop'].iloc[0]
    sp_rows.append(i)
    sp_cols.append(pop_start)
    v.append(pop)
    
    i += 1

test_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(x_test), l), dtype=float)

In [9]:
sp.save_npz(os.path.join(dir_, folder, 'pop_d_topN_test_x_' + file_name), test_x)

In [7]:
test_x = sp.load_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name + '.npz'))

In [8]:
print(test_x[1,:])

  (0, 0)	1.0
  (0, 232)	1.0
  (0, 39356)	1.0
  (0, 39374)	6.0


In [9]:
order = 4
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.000001,
    reg=0.0001,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)
predictions = model.predict(test_x)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


100%|██████████| 10/10 [00:10<00:00,  1.10s/epoch]


In [10]:
len(predictions), predictions

(8169189,
 array([0.07101551, 0.023108  , 0.14877048, ..., 0.02066167, 0.02337948,
        0.0207149 ], dtype=float32))

In [11]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', predictions, False) 
df2[:10]

NameError: name 'x_test' is not defined

In [14]:
# df2.to_pickle(os.path.join(dir_, folder, 'pop_d_topN_pred_' + file_name[:-3] + 'pkl'))

In [ ]:
predictions = model.predict(test_x_sp)
print('RMSE: {}'.format(math.sqrt(mean_squared_error(test_y_sp, predictions))))
print('MAE: {}'.format(mean_absolute_error(test_y_sp, predictions)))

In [ ]:
predictions = model.predict(train_x)
print('RMSE: {}'.format(math.sqrt(mean_squared_error(train_y, predictions))))
print('MAE: {}'.format(mean_absolute_error(train_y, predictions)))

In [ ]:
predictions = df2

In [ ]:
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:10]

In [ ]:
num_user = len(predictions['uid'].unique())
num_user

In [ ]:
# Personal Recommendation
n=100
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)

In [ ]:
train.sort_values(by=['uid','tid'])[:10]